**анализ книжного магазина**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.


Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель исследования** - изучить данные о книгах, авторах, издательствах, рейтингах и обзорах и ответить на поставленные вопросы 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
# список таблиц
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

#просмотр типов данных
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 0"
    df = pd.io.sql.read_sql(sql=text(query), con = con)
    print(f"Columns in {table} table:")
    display(df.dtypes)


Columns in books table:


book_id             object
author_id           object
title               object
num_pages           object
publication_date    object
publisher_id        object
dtype: object

Columns in authors table:


author_id    object
author       object
dtype: object

Columns in publishers table:


publisher_id    object
publisher       object
dtype: object

Columns in ratings table:


rating_id    object
book_id      object
username     object
rating       object
dtype: object

Columns in reviews table:


review_id    object
book_id      object
username     object
text         object
dtype: object

In [3]:

for table in tables:
    query = f"SELECT * FROM {table}"
    df = pd.io.sql.read_sql(sql=text(query), con = con)

    print(f"Data in {table} table:")
    display(df.head())


Data in books table:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Data in authors table:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Data in publishers table:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Data in ratings table:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Data in reviews table:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [4]:
# создаем функцию для выполнения SQL-запросов
def execute_query(query):
    return pd.io.sql.read_sql(sql=text(query), con=engine.connect())

# Посчитайте, сколько книг вышло после 1 января 2000 года
query = '''SELECT COUNT(*) FROM books WHERE publication_date > '2000-01-01';'''
display(execute_query(query))

,count
0,819


после 1 января 2000 года было впушенно 819 книг

Сначала код не работал и я прописал функцию в каждом блоке кода, но потом без этого заработало, если что можно раскоментировать 

In [5]:
# колличество и средняя оценка 
# def execute_query(query):
#     return pd.io.sql.read_sql(sql=text(query), con=engine.connect())

query = '''
SELECT 
    books.book_id,
    books.title, 
    COUNT(DISTINCT review_id) as review_count, 
    AVG(rating) as avg_rating
FROM 
    books 
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY 
    books.book_id,
    books.title
'''

display(execute_query(query))

,book_id,title,review_count,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


таблица со средними оценками книн 

In [6]:
# def execute_query(query):
#     return pd.io.sql.read_sql(sql=text(query), con=engine.connect())


# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц
query = '''
SELECT 
    publishers.publisher, 
    COUNT(books.book_id) as book_count
FROM 
    books 
INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE 
    books.num_pages > 50
GROUP BY 
    publishers.publisher
ORDER BY 
    book_count DESC
LIMIT 1
'''
display(execute_query(query))


,publisher,book_count
0,Penguin Books,42


издательство Penguin Books выпостило 42 киги более 50 страниц 

In [7]:
# def execute_query(query):
#     return pd.io.sql.read_sql(sql=text(query), con=engine.connect())

# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
query = '''
SELECT 
    authors.author, 
    AVG(avg_book_rating) as avg_author_rating
FROM 
    (SELECT 
        books.author_id, 
        AVG(ratings.rating) as avg_book_rating
    FROM 
        books 
    INNER JOIN ratings ON books.book_id = ratings.book_id
    GROUP BY 
        books.book_id
    HAVING 
        COUNT(ratings.rating) >= 50) as subquery
INNER JOIN authors ON subquery.author_id = authors.author_id
GROUP BY 
    authors.author
ORDER BY 
    avg_author_rating DESC
LIMIT 1
'''
display(execute_query(query))



,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.283844


автор с самой высокой среденей оценкой 4.28 J.K. Rowling/Mary GrandPré

In [8]:
# def execute_query(query):
#     return pd.io.sql.read_sql(sql=text(query), con=engine.connect())
# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок
query = '''
SELECT 
    AVG(review_count) as avg_review_count
FROM 
    (SELECT 
        username, 
        COUNT(review_id) as review_count
    FROM 
        reviews 
    WHERE 
        username IN (SELECT 
                        username 
                     FROM 
                        ratings 
                     GROUP BY 
                        username 
                     HAVING 
                        COUNT(rating_id) > 48)
    GROUP BY 
        username) as subquery
'''
display(execute_query(query))

,avg_review_count
0,24.0


среднее количество обзоров от пользователей, которые поставили больше 48 оценок 24

In [9]:

#def execute_query(query):
    #return pd.io.sql.read_sql(sql=text(query), con=engine.connect())
# подсчет количества строк в каждой таблице
for table in tables:
    query = f'SELECT COUNT(*) FROM {table};'
    result = execute_query(query)
    print(f'Количество строк в таблице {table}: {result.values}')

Количество строк в таблице books: [[1000]]
Количество строк в таблице authors: [[636]]
Количество строк в таблице publishers: [[340]]
Количество строк в таблице ratings: [[6456]]
Количество строк в таблице reviews: [[2793]]


**Вывод**


С 1 января 2000 года было выпущено 819 книг. Издательство Penguin Books выпустило 42 книги, каждая из которых содержит более 50 страниц. Автор с самой высокой средней оценкой 4.28 - это Rowling/Mary GrandPré Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24

колличество строк 
- Количество строк в таблице книг: **1000**
- Количество строк в таблице авторов: **636**
- Количество строк в таблице издателей: **340**
- Количество строк в таблице рейтингов: **6456**
- Количество строк в таблице обзоров: **2793**